<a href="https://colab.research.google.com/github/sgparrot/sgparrot2021Lv1-1/blob/main/Simpsons/simpson_preprocessing_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import time
import glob
import zipfile

import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import albumentations as A
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical, Sequence

from pprint import pprint
from datetime import datetime
from sklearn.model_selection import train_test_split

%matplotlib inline

# 데이터 불러오기

In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
target=os.listdir("/content/drive/MyDrive/parrot/simpson/simpsons_dataset/train")
target.sort()

In [4]:
category={}
for i, name in enumerate(target):
  category[i]=name

pprint(category)

{0: 'bart',
 1: 'charles_montgomery',
 2: 'krusty',
 3: 'lisa_simpson',
 4: 'marge_simpson',
 5: 'milhouse_van_houten',
 6: 'moe_szyslak',
 7: 'ned_flanders',
 8: 'principal_skinner'}


In [5]:
train_data=np.load("/content/drive/MyDrive/parrot/simpson/train_224_npz.npz")
test_data=np.load("/content/drive/MyDrive/parrot/simpson/test_224_npz.npz")

In [6]:
X_train=train_data['X_train_npz']
y_train=train_data['y_train_npz']

y_train=to_categorical(y_train, len(target))

X_test=test_data['X_test_npz']
y_test_list=pd.read_csv("/content/drive/MyDrive/parrot/simpson/test_sample.csv", index_col=0)
y_test=np.array(y_test_list['files'])

In [7]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(11248, 224, 224, 3) (11248, 9) (1086, 224, 224, 3) (1086,)


# train test split

In [8]:
train_data, val_data, train_labels, val_labels = train_test_split(X_train, y_train, test_size=0.2)

# augmentation

In [9]:
train_batch_size = 64
test_batch_size = 64
image_size = 224

In [10]:
# train data augmentor
augmentor_train = A.Compose([
                             A.HorizontalFlip(p=0.5),
                             A.ShiftScaleRotate(p=0.75),
                             A.RandomBrightnessContrast(
                                 brightness_limit=(-0.25, 0.25),
                                 contrast_limit=(-0.25, 0.25), p=1
                                 ),   # 밝기와 대비 조절
                             A.CLAHE(p=1),   # Contrast-Limited Adaptive Histogram Equalization
                             A.OneOf([A.MotionBlur(p=1),   # Blur
                                      A.OpticalDistortion(p=1),   # 왜곡
                                      A.GaussNoise(p=1)], p=0.5),   # Noise
                             A.Cutout(num_holes = 32, max_h_size=4,
                                      max_w_size=4, p=0.75),   # dropout
                             A.Normalize(mean=(0, 0, 0), std=(1, 1, 1))
])

# test data augmentor
augmentor_test = A.Compose([A.Normalize(mean=(0, 0, 0), std=(1, 1, 1))])

# Dataloader

In [11]:
class Dataloader(Sequence): # Sequence 상속 받는 클래스 정의

    def __init__(self, x_set, y_set, batch_size,
                 augmentor= None, shuffle=False):   # augmentor가 추가됨

        """
        x_set, y_set: train set or test set (np.ndarray)
        batch_size: the size of data when a batch is enumerated (int)
        augmentor: data augmentor (albumantation, imgaug, ...)
        shuffle: whether shuffle the data or not after each epoch (boolean)
        """

        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.shuffle=shuffle
        self.on_epoch_end()

    def __len__(self):  #전체 길이, #data/batch_size
        return int(np.ceil(len(self.x) / self.batch_size))

    def __getitem__(self, idx): #batch index 값에 따라 데이터 반환
        indices = self.indices[idx*self.batch_size: (idx+1)*self.batch_size]

        batch_x = [self.x[i] for i in indices]
        batch_y = [self.y[i] for i in indices]

        for image_index in range(indices.shape[0]):

            image = batch_x[image_index]

            if self.augmentor is not None:
                image = self.augmentor(image=image)["image"]

            batch_x[image_index] = image
        
        return np.array(batch_x), np.array(batch_y)
    
    ### Epoch이 한 번 끝날 때마다 shuffle
    def on_epoch_end(self):
        self.indices = np.arange(len(self.x))
        if self.shuffle == True:
            np.random.shuffle(self.indices)

In [12]:
train_loader = Dataloader(train_data, train_labels, batch_size=train_batch_size,
                          augmentor=augmentor_train, shuffle=True)
val_loader = Dataloader(val_data, val_labels, batch_size=train_batch_size,
                        augmentor=augmentor_train, shuffle=True)
test_loader = Dataloader(X_test, y_test, batch_size=test_batch_size,
                         augmentor=augmentor_test, shuffle=False)

In [13]:
examples = enumerate(train_loader)
batch_idx, (example_data, example_target) = next(examples)

print("Data  : {}".format(example_data.shape))
print("Target: {}".format(example_target.shape))

Data  : (64, 224, 224, 3)
Target: (64, 9)


In [14]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_target) = next(examples)

print("Target: {}".format(example_target.shape))
print("Data  : {}".format(example_data.shape))

Target: (64,)
Data  : (64, 224, 224, 3)


# modeling

In [15]:
model=tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False,
                                        weights='imagenet', pooling='avg')
model.trainable=False

74842112/74836368 [==============================] - 3s 0us/step


In [16]:
input=model.input
x=tf.keras.layers.Dense(128, activation='relu')(model.output)
output=tf.keras.layers.Dense(9, activation='softmax')(x)
Model=tf.keras.Model(inputs=input, outputs=output)

In [17]:
Model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [18]:
Model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
from tensorflow.keras.callbacks import EarlyStopping
earlystop=EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode="auto", baseline=None, restore_best_weights=True)

In [20]:
history=Model.fit(train_loader, batch_size=train_batch_size, epochs=100, validation_data=val_loader, verbose=1, callbacks=[earlystop])

Epoch 1/100
141/141 [==============================] - 185s 985ms/step - loss: 1.3862 - accuracy: 0.5301 - val_loss: 0.9408 - val_accuracy: 0.7080
Epoch 2/100
141/141 [==============================] - 127s 900ms/step - loss: 0.8321 - accuracy: 0.7437 - val_loss: 0.8060 - val_accuracy: 0.7409
Epoch 3/100
141/141 [==============================] - 127s 899ms/step - loss: 0.7105 - accuracy: 0.7723 - val_loss: 0.7154 - val_accuracy: 0.7658
Epoch 4/100
141/141 [==============================] - 127s 900ms/step - loss: 0.6423 - accuracy: 0.7964 - val_loss: 0.7007 - val_accuracy: 0.7773
Epoch 5/100
141/141 [==============================] - 126s 895ms/step - loss: 0.6216 - accuracy: 0.8054 - val_loss: 0.6159 - val_accuracy: 0.8036
Epoch 6/100
141/141 [==============================] - 127s 898ms/step - loss: 0.5858 - accuracy: 0.8103 - val_loss: 0.6180 - val_accuracy: 0.7964
Epoch 7/100
141/141 [==============================] - 127s 901ms/step - loss: 0.5520 - accuracy: 0.8254 - val_loss: 0